In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:
import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_sellers = read_file(spark,"parquet",f'{bronze}/sellers')

In [0]:
df_dedup = deduplicate(df_sellers,["seller_id"])


df_stand = standardize_string_columns(df_dedup,{"seller_city":"lower_trim","seller_state":"upper_trim"})

columns = {
        "created_at": {"type": "timestamp"},
        "updated_at": {"type": "timestamp"},
        "source": {"type": "string", "default": "sellers"}}

df_final_standardize = enrich_add_col(df_stand, columns)

# df_final_standardize.display()

In [0]:
# Write data delta file /delta table in silver location

write_file(df_final_standardize,f"{silver}/silver_sellers")
write_delta_liquid(df_final_standardize, f"{catalog}.{schema}.silver_sellers",["seller_id"])